In [ ]:
import svetlanna as sv
from svetlanna import Wavefront, SimulationParameters
from svetlanna.parameters import OptimizableFloat, OptimizableTensor
from svetlanna.wavefront import mul
from svetlanna.units import ureg
import torch

# Пример создания нового оптического элемента

Рассмотрим оптический элемент, который действует на волновой фронт следующим образом:
$$
f(u) = a\left(\hat{W}u\right)^b
$$
где $u$ - падающий волновой фронт, и $a$ , $b$ - некоторые параметры. $W$ - это двумерная маска пропускания в плоскости (x,y).
Произведение $\hat{W}x$ представляет собой поэлементное умножение:
$$\left[\hat{W}u\right](x_i, y_i) = \hat{W}(x_i, y_i)u(x_i, y_i)$$

Для создания класса, представляющего собой новый оптический элемент с функцией пропускания, описанной выше, необходимо наследовать этот класса от родительского класса `svetlanna.elements.Element`

In [ ]:
class MyElement(sv.elements.Element):
    def __init__(
        self,
        simulation_parameters: SimulationParameters,
        a: OptimizableFloat,
        b: int,
        W: OptimizableTensor
    ) -> None:
        super().__init__(simulation_parameters)  # эта строчка обязательно должна быть  в конструкторе!

        self.a = self.process_parameter(
            'a', a
        )
        self.b = self.process_parameter(
            'b', b
        )
        self.W = self.process_parameter(
            'W', W
        )

    def forward(self, incident_wavefront: Wavefront) -> Wavefront:
        r = mul(
            incident_wavefront,
            self.W,
            ('H', 'W'),
            self.simulation_parameters
        )
        return self.a * r**self.b

Рассмотрим подробнее каждую часть кода:

Для явного указания объектов, который необходимо оптимизировать, доступны типы `OptimizableFloat` и `OptimizableTensor`
* `OptimizableFloat` используется для скалярных значений
* `OptimizableTensor` используются для тензоров - векторов и матриц

Используемые параметры(тензоры или скаляры) должны быть зарегистрированы с использованием метода `process_parameter`.
Этот метод выполняет различные действия в зависимости от предоставленного аргумента:

* если аргумент требует вычисления градиента(например `torch.nn.Parameter`), он регистрирует параметр в экземпляре `torch.nn.Module` 
* если аргумент является тензором, он регистрирует его как буфер
* в остальных случая метод не совершает никаких операций

**Лучшая практика**: всегда использовать метод `process_parameter` для любого аргумента, переданного в конструктор.

Метод `forward` должен быть применен к любому новому элементу
Помимо умножения и возведения в степень, он включает поэлементное произведение.
Для осуществления поэлементного умножения между волновым фронтом и тензором необходимо использовать функцию `mul`.
Для выполнения произведения необходимо указать оси $W$.
В этом примере $W$ - это двумерная маска пропускания в плоскости (x,y), соответственно его оси указаны как `('H', 'W')`
Порядок имен осей должен соответствовать порядку осей тензора при вызове `.shape`

Подход с использованием метода `mul` обеспечивает совместимость с дальнейшими изменениями порядка осей волнового фронта и изменениями количества осей (например, ось партии, физические свойства, такие как `поляризация`, `длина волны` и т. д.).

In [ ]:
sim_params = SimulationParameters({
    'W': torch.linspace(-1, 1, 10) * ureg.cm,
    'H': torch.linspace(-1, 1, 10) * ureg.cm,
    'wavelength': torch.tensor([400, 500, 600]) * ureg.nm
})

wf = Wavefront(torch.rand((3, 10, 10)))

In [ ]:
el1 = MyElement(
    sim_params,
    a=sv.ConstrainedParameter(2., min_value=0., max_value=5),
    b=2,
    W=torch.rand((10, 10))
)

print(el1(wf).shape)  # transmitted wavefront shape

torch.Size([3, 10, 10])


# Свойства, зависящие от длины волны

Рассмотрим оптический элемент с нелинейной функцией пропускания, которая дана выражением
$$
f(u) = u \frac{600}{600 + \lambda}
$$
где $\lambda$ - длина волны в нм.

In [ ]:
class MyNonlinearElement(sv.elements.Element):
    def forward(self, incident_wavefront: Wavefront) -> Wavefront:
        lmbda = self.simulation_parameters.axes.wavelength / ureg.nm
        t = 600 / (600 + lmbda)
        return mul(
            incident_wavefront,
            t,
            'wavelength',
            self.simulation_parameters
        )

В этом примере реализован только метод `forward`.
Для построения $f(u)$ длина волны должна быть получена из параметров моделирования.
Общий подход к доступу к любой оси из параметров моделирования: `self.simulation_parameters.axes.<имя оси>`.
В этом случае длина волны извлекается с помощью `self.simulation_parameters.axes.wavelength`.
Затем для умножения используется функция `mul`.
Это гарантирует работоспособность кода в различных сценариях, например: когда в параметрах моделирования указана одна длина волны (`'wavelength': 500 * ureg.nm`) или когда добавлено больше осей.
В этих случаях тот же код будет работать без каких-либо изменений.

In [ ]:
el2 = MyNonlinearElement(
    sim_params
)

print(el2(wf).shape)  # transmitted wavefront shape

torch.Size([3, 10, 10])


# Свойства, зависящие от длины волны, улучшение

Можно заметить, что `t` пересчитывается каждый раз при вызове метода `forward`.
Для сокращения вычислений эту переменную можно вычислить один раз во время инициализации и зарегистрировать как буфер.

In [ ]:
class MyNonlinearElementImproved(sv.elements.Element):
    def __init__(self, simulation_parameters: SimulationParameters) -> None:
        super().__init__(simulation_parameters)

        lmbda = self.simulation_parameters.axes.wavelength / ureg.nm
        t = 600 / (600 + lmbda)

        if isinstance(lmbda, torch.Tensor):
            self.t = self.make_buffer('t', t)
        else:
            self.t = t

    def forward(self, incident_wavefront: Wavefront) -> Wavefront:
        return mul(
            incident_wavefront,
            self.t,
            'wavelength',
            self.simulation_parameters
        )

Для того, чтобы сохранить любой тензор в качестве буфера, следует использовать метод `make_buffer`.
Этот метод применим только к объектам `torch.Tensors`, поэтому необходимо условие `if`.

Важно отметить, что если выражение зависит от параметра, требующего вычисления градиента, тензор не следует буферизировать.
Его необходимо вычислять при каждом вызове метода forward для обеспечения корректного отслеживания градиента.

# Спецификации

In [ ]:
from svetlanna.specs import ReprRepr, PrettyReprRepr, NpyFileRepr, ImageRepr
from svetlanna.specs import ParameterSpecs

В настоящее время существует четыре типа представления переменных: `ReprRepr`, `PrettyReprRepr`, `NpyFileRepr`, `ImageRepr`.

Для отображения спецификаций (и их последующего сохранения) необходимо реализовать метод `to_specs`.
Для представлений требуется аргумент `value`, которым могут быть любые релевантные данные, например, `self.mask.abs()`.
В следующем примере эта концепция демонстрируется на основе случайных тензоров.

In [ ]:
class ElementWithSpecs(sv.elements.Element):
    def forward(self, incident_wavefront: Wavefront) -> Wavefront:
        ...

    def to_specs(self) -> list[ParameterSpecs]:
        return [
            ParameterSpecs(
                'a', (
                    ReprRepr(123),
                    PrettyReprRepr(123, units='cm'),
                )
            ),
            ParameterSpecs(
                'b', (
                    ImageRepr(
                        torch.rand((100, 100)).numpy(force=True)
                    ),
                    NpyFileRepr(
                        torch.rand((100, 100)).numpy(force=True)
                    ),
                    PrettyReprRepr(
                        torch.rand((100, 100)), units='nm'),
                )
            ),
            ParameterSpecs(
                'c', (
                    ImageRepr(
                        torch.rand((100, 100)).numpy(force=True)
                    ),
                )
            )
        ]

In [ ]:
# specs can be displayed in the jupyter cell
ElementWithSpecs(sim_params)

ElementWithSpecs()